In [116]:
import pandas as pd
import seaborn as sns
import sqlite3
import os
from IPython.display import HTML
import numpy as np

In [5]:
conn = sqlite3.connect('nba.sqlite')
cursor = conn.cursor()
conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
output_folder = "csv"
os.makedirs(output_folder, exist_ok=True)

In [6]:
# Print the table names
# for table in tables:
#     table_name = table[0]
#     # Read the table data into a pandas DataFrame
#     df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
#     output_file = os.path.join(output_folder, f"{table_name}.csv")
#     # Save DataFrame to CSV
#     df.to_csv(output_file, index=False)
# conn.close()

In [7]:
folder_path = "csv/"

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Print the file names
for file in files:
    print(file)


common_player_info.csv
draft_combine_stats.csv
draft_history.csv
game.csv
game_info.csv
game_summary.csv
inactive_players.csv
line_score.csv
officials.csv
other_stats.csv
player.csv
play_by_play.csv
team.csv
team_details.csv
team_history.csv
team_info_common.csv


In [226]:
# "play_by_play.csv"
file_names = ["common_player_info.csv","play_by_play.csv", "draft_combine_stats.csv","draft_history.csv","game.csv","game_info.csv","game_summary.csv","inactive_players.csv","line_score.csv","officials.csv","other_stats.csv","player.csv","team.csv","team_details.csv","team_history.csv","team_info_common.csv"]

In [227]:
dataframes = {}
for file_name in file_names:
    file_path = "csv/" + file_name  # Update with the actual folder path
    df = pd.read_csv(file_path, low_memory=False)
    dataframes[file_name] = df

In [229]:
df_common_player_info = dataframes["common_player_info.csv"]
df_draft_combine_stats = dataframes["draft_combine_stats.csv"]
df_draft_history = dataframes["draft_history.csv"]
df_game= dataframes["game.csv"]
df_game_info = dataframes["game_info.csv"]
df_game_summary = dataframes["game_summary.csv"]
df_inactive_players = dataframes["inactive_players.csv"]
df_line_score = dataframes["line_score.csv"]
df_officials = dataframes["officials.csv"]
df_other_stats = dataframes["other_stats.csv"]
df_player = dataframes["player.csv"]
df_play_by_play = dataframes["play_by_play.csv"]
df_team = dataframes["team.csv"]
df_team_details = dataframes["team_details.csv"]
df_team_history = dataframes["team_history.csv"]
df_team_info_common = dataframes["team_info_common.csv"]

## Data cleaning: 
df_team, df_team_details, df_team_history, df_team_info_common

In [11]:
#checking data types
#df_team
df_team["year_founded"] = df_team["year_founded"].astype(int)
df_team.loc[0,"state"] = "Georgia"

# df_team_details
df_team_details[["yearfounded", "arenacapacity"]] = df_team_details[["yearfounded", "arenacapacity"]].astype(pd.Int64Dtype())
new_column_names= {"yearfounded" : "year_founded","arenacapacity": "arena_capacity", "dleagueaffiliation":"league_affliation"}
df_team_details = df_team_details.rename(columns=new_column_names)
df_team_details.info()
html = (df_team_details.to_html(render_links=True, escape=False))
HTML(html)

# df_team_info_common.info()
# df_team_history


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   team_id            30 non-null     int64 
 1   abbreviation       30 non-null     object
 2   nickname           30 non-null     object
 3   year_founded       30 non-null     Int64 
 4   city               30 non-null     object
 5   arena              30 non-null     object
 6   arena_capacity     20 non-null     Int64 
 7   owner              30 non-null     object
 8   generalmanager     30 non-null     object
 9   headcoach          30 non-null     object
 10  league_affliation  30 non-null     object
 11  facebook           30 non-null     object
 12  instagram          30 non-null     object
 13  twitter            30 non-null     object
dtypes: Int64(2), int64(1), object(11)
memory usage: 3.5+ KB


,team_id,abbreviation,nickname,year_founded,city,arena,arena_capacity,owner,generalmanager,headcoach,league_affliation,facebook,instagram,twitter
0,1610612737,ATL,Hawks,1949,Atlanta,State Farm Arena,18729,Tony Ressler,Travis Schlenk,Quin Snyder,College Park Skyhawks,https://www.facebook.com/hawks,https://instagram.com/atlhawks,https://twitter.com/ATLHawks
1,1610612738,BOS,Celtics,1946,Boston,TD Garden,18624,Wyc Grousbeck,Brad Stevens,Joe Mazzulla,Maine Celtics,https://www.facebook.com/bostonceltics,https://instagram.com/celtics,https://twitter.com/celtics
2,1610612739,CLE,Cavaliers,1970,Cleveland,Rocket Mortgage FieldHouse,20562,Dan Gilbert,Koby Altman,JB Bickerstaff,Cleveland Charge,https://www.facebook.com/Cavs,https://instagram.com/cavs,https://twitter.com/cavs
3,1610612740,NOP,Pelicans,2002,New Orleans,Smoothie King Center,,Tom Benson,Trajan Langdon,Willie Green,Birmingham Squadron,https://www.facebook.com/PelicansNBA,https://instagram.com/pelicansnba,https://twitter.com/PelicansNBA
4,1610612741,CHI,Bulls,1966,Chicago,United Center,21711,Jerry Reinsdorf,Arturas Karnisovas,Billy Donovan,Windy City Bulls,https://www.facebook.com/chicagobulls,https://instagram.com/chicagobulls,https://twitter.com/chicagobulls
5,1610612742,DAL,Mavericks,1980,Dallas,American Airlines Center,19200,Mark Cuban,Nico Harrison,Jason Kidd,Texas Legends,https://www.facebook.com/dallasmavs,https://instagram.com/dallasmavs,https://twitter.com/dallasmavs
6,1610612743,DEN,Nuggets,1976,Denver,Ball Arena,,Stan Kroenke,Calvin Booth,Michael Malone,Grand Rapids Gold,https://www.facebook.com/DenverNuggets,https://instagram.com/nuggets,https://twitter.com/nuggets
7,1610612744,GSW,Warriors,1946,Golden State,Chase Center,,Joe Lacob,Bob Myers,Steve Kerr,Santa Cruz Warriors,https://www.facebook.com/warriors,https://instagram.com/warriors,https://twitter.com/warriors
8,1610612745,HOU,Rockets,1967,Houston,Toyota Center,18104,Tilman Fertitta,Rafael Stone,Stephen Silas,Rio Grande Valley Vipers,https://www.facebook.com/houstonrockets,https://instagram.com/houstonrockets,https://twitter.com/HoustonRockets
9,1610612746,LAC,Clippers,1970,Los Angeles,Crypto.com Arena,19060,Steve Ballmer,Michael Winger,Tyronn Lue,Agua Caliente Clippers of Ontario,https://www.facebook.com/LAClippers,https://instagram.com/laclippers,https://twitter.com/LAClippers


In [12]:
#checking the null values
# df_team.isnull().sum()
# df_team_details.isnull().sum()
# df_team_history.isnull().sum()
# df_team_info_common.isnull().sum()

# df_team_history.dropna(axis="index", how="all")

In [13]:
df_team.rename(columns={"id":"team_id"}, inplace=True)
df_team.head()

,team_id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [14]:
df_filtered = df_team_history[df_team_history["city"] == "Charlotte"]
df_filtered

,team_id,city,nickname,year_founded,year_active_till
57,1610612766,Charlotte,Hornets,2014,2019
58,1610612766,Charlotte,Bobcats,2004,2013
59,1610612766,Charlotte,Hornets,1988,2001


### Joining All The Teams Dataframes

In [15]:
join_team_table_1 = df_team.merge(df_team_details, on =["team_id","abbreviation", "nickname","city", "year_founded"], how = "inner")
join_team_table_2 = join_team_table_1.merge(df_team_info_common, on =["team_id"], how = "inner")
join_team_table_2.drop(columns=["team_city","team_name", "team_abbreviation", "team_code","team_slug"], inplace=True)
join_team_table_2.head(10)

,team_id,full_name,abbreviation,nickname,city,state,year_founded,arena,arena_capacity,owner,...,league_id,season_id,pts_rank,pts_pg,reb_rank,reb_pg,ast_rank,ast_pg,opp_pts_rank,opp_pts_pg
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949,State Farm Arena,18729,Tony Ressler,...,0,22022,7,117.0,12,43.7,20,24.6,21,116.9
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946,TD Garden,18624,Wyc Grousbeck,...,0,22022,4,117.7,6,45.2,7,26.3,9,112.4
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970,Rocket Mortgage FieldHouse,20562,Dan Gilbert,...,0,22022,25,112.0,23,41.7,18,24.9,1,106.4
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002,Smoothie King Center,<NA>,Tom Benson,...,0,22022,16,114.2,14,43.7,11,25.5,16,113.4
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966,United Center,21711,Jerry Reinsdorf,...,0,22022,20,113.0,21,42.6,22,24.1,11,112.7
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980,American Airlines Center,19200,Mark Cuban,...,0,22022,17,114.0,30,38.5,29,22.4,14,113.2
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976,Ball Arena,<NA>,Stan Kroenke,...,0,22022,5,117.1,16,43.2,2,29.3,10,112.6
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946,Chase Center,<NA>,Joe Lacob,...,0,22022,3,118.0,9,44.2,1,29.6,22,117.1
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967,Toyota Center,18104,Tilman Fertitta,...,0,22022,29,110.3,4,46.4,28,22.7,27,118.2
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970,Crypto.com Arena,19060,Steve Ballmer,...,0,22022,22,112.7,15,43.4,25,23.2,13,113.0


In [16]:
join_team_table_3 = join_team_table_2.merge(df_team_history, on =["team_id", "nickname","city", "year_founded"], how = "left")
join_team_table_3.head()

,team_id,full_name,abbreviation,nickname,city,state,year_founded,arena,arena_capacity,owner,...,season_id,pts_rank,pts_pg,reb_rank,reb_pg,ast_rank,ast_pg,opp_pts_rank,opp_pts_pg,year_active_till
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949,State Farm Arena,18729,Tony Ressler,...,22022,7,117.0,12,43.7,20,24.6,21,116.9,NaN
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946,TD Garden,18624,Wyc Grousbeck,...,22022,4,117.7,6,45.2,7,26.3,9,112.4,2019.0
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970,Rocket Mortgage FieldHouse,20562,Dan Gilbert,...,22022,25,112.0,23,41.7,18,24.9,1,106.4,2019.0
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002,Smoothie King Center,<NA>,Tom Benson,...,22022,16,114.2,14,43.7,11,25.5,16,113.4,NaN
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966,United Center,21711,Jerry Reinsdorf,...,22022,20,113.0,21,42.6,22,24.1,11,112.7,2019.0


### game, game_info, game_summary

## Data Cleaning

In [50]:
#df_game
# df_game["game_date"] = pd.to_datetime(df_game["game_date"])

#df_game_info
# df_game_info["game_date"] = pd.to_datetime(df_game_info["game_date"])

#df_game_summary
# df_game_summary["game_date_est"] = pd.to_datetime(df_game_summary["game_date_est"])
# df_game_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60796 entries, 0 to 60795
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   game_date_est                     60796 non-null  datetime64[ns]
 1   game_sequence                     31611 non-null  float64       
 2   game_id                           60796 non-null  int64         
 3   game_status_id                    60796 non-null  int64         
 4   game_status_text                  31069 non-null  object        
 5   gamecode                          60796 non-null  object        
 6   home_team_id                      60796 non-null  int64         
 7   visitor_team_id                   60796 non-null  int64         
 8   season                            60796 non-null  int64         
 9   live_period                       60796 non-null  int64         
 10  live_pc_time                      1748 non-nul

In [52]:
df_game_summary["live_pc_time"].isnull().sum()

59048

In [60]:
# df_game.info()
# df_game_info.info()
# df_game_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62367 entries, 0 to 62366
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   season_id               62367 non-null  int64         
 1   team_id_home            62367 non-null  int64         
 2   team_abbreviation_home  62367 non-null  object        
 3   team_name_home          62367 non-null  object        
 4   game_id                 62367 non-null  int64         
 5   game_date               62367 non-null  datetime64[ns]
 6   matchup_home            62367 non-null  object        
 7   wl_home                 62360 non-null  object        
 8   min                     62367 non-null  int64         
 9   fgm_home                62349 non-null  float64       
 10  fga_home                44505 non-null  float64       
 11  fg_pct_home             44466 non-null  float64       
 12  fg3m_home               46714 non-null  float6

In [68]:
# df_game.isnull().sum()
# df_game_info.isnull().sum()
# df_game_summary.isnull().sum()

game_date_est                           0
game_sequence                       29185
game_id                                 0
game_status_id                          0
game_status_text                    29727
gamecode                                0
home_team_id                            0
visitor_team_id                         0
season                                  0
live_period                             0
live_pc_time                        59048
natl_tv_broadcaster_abbreviation    56135
live_period_time_bcast                  0
wh_status                               0
dtype: int64

## Joining(game, game_info, game_summary)

In [78]:
# df_game
# df_game_info
# df_game_summary

In [83]:
join_game_table1 = df_game_info.merge(df_game_summary, on=["game_id"], how="inner")
join_game_table2 = df_game.merge(join_game_table1, on=["game_id", "game_date"], how="left")

join_game_table2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 62368 entries, 0 to 62367
Data columns (total 69 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   season_id                         62368 non-null  int64         
 1   team_id_home                      62368 non-null  int64         
 2   team_abbreviation_home            62368 non-null  object        
 3   team_name_home                    62368 non-null  object        
 4   game_id                           62368 non-null  int64         
 5   game_date                         62368 non-null  datetime64[ns]
 6   matchup_home                      62368 non-null  object        
 7   wl_home                           62361 non-null  object        
 8   min                               62368 non-null  int64         
 9   fgm_home                          62350 non-null  float64       
 10  fga_home                          44506 non-nu

## Data Cleaning (player, common_player_info, inactive_players)

In [102]:
mask = df_player["first_name"].isnull()
result = df_player[mask]

print(result)



           id     full_name first_name     last_name  is_active
1666     2775  Ha Seung-jin        NaN  Ha Seung-jin          0
3129     2403          Nene        NaN          Nene          0
4140   201180       Sun Yue        NaN       Sun Yue          0
4789     2397      Yao Ming        NaN      Yao Ming          0
4795   201146   Yi Jianlian        NaN   Yi Jianlian          0
4821  1627753       Zhou Qi        NaN       Zhou Qi          0


In [130]:
df_inactive_players["jersey_num"].unique()

array([14, 41,  8, 44,  3, 53, 54, 45, 55, 20,  4, 32,  7, 24,  1, 33, 52,
       31,  5, 48, 50,  6, 34, 30, 51, 17, 10, 42, 11, 23, 25, 35, 13, 12,
       43, 15,  9, 21, 73,  0,  2, 40, 49, 29, 16, 18, 77, 91, 62, 19, 22,
       99, 38, -1, 27, 47, 26, 56, 90, 36, 88, 28, 93, 66, 84, 98, 97, 96,
       37, 72, 86, 85, 92, 58, 39, 89, 63, 95, 46, 67, 71, 81, 94, 57])

In [268]:
#df_player

df_player= df_player.rename(columns={"id":"player_id"})
df_player.info()
#df_inactive_players
# df_inactive_players["jersey_num"] = df_inactive_players["jersey_num"].replace('-', np.nan).astype(float)
# df_inactive_players["jersey_num"] = df_inactive_players["jersey_num"].fillna(-1).astype(int)

# df_inactive_players.info()
df_common_player_info.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4831 entries, 0 to 4830
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   player_id   4831 non-null   int64 
 1   full_name   4831 non-null   object
 2   first_name  4825 non-null   object
 3   last_name   4831 non-null   object
 4   is_active   4831 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 188.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2457 entries, 0 to 2456
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   person_id                         2457 non-null   int64  
 1   first_name                        2457 non-null   object 
 2   last_name                         2457 non-null   object 
 3   display_first_last                2457 non-null   object 
 4   display_last_comma_first          2457 non-null   object 
 5   display_f

In [269]:

df_common_player_info = df_common_player_info.rename(columns={"person_id" : "player_id", "jersey": "jersey_num"})

df_common_player_info["birthdate"] = pd.to_datetime(df_common_player_info["birthdate"])
df_common_player_info["weight"] = df_common_player_info["weight"].replace("nan", 0)
# df_common_player_info[["first_name","jersey_num"]]
# df_common_player_info["position"].unique()
df_common_player_info.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2457 entries, 0 to 2456
Data columns (total 33 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   player_id                         2457 non-null   int64         
 1   first_name                        2457 non-null   object        
 2   last_name                         2457 non-null   object        
 3   display_first_last                2457 non-null   object        
 4   display_last_comma_first          2457 non-null   object        
 5   display_fi_last                   2457 non-null   object        
 6   player_slug                       2457 non-null   object        
 7   birthdate                         2457 non-null   datetime64[ns]
 8   school                            2453 non-null   object        
 9   country                           2457 non-null   object        
 10  last_affiliation                  2457 non-null 

### Joning(player, inactive_players, common_player_info)



In [214]:
# df_player
# df_common_player_info
df_inactive_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105217 entries, 0 to 105216
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   game_id            105217 non-null  int64 
 1   player_id          105217 non-null  int64 
 2   first_name         105216 non-null  object
 3   last_name          105216 non-null  object
 4   jersey_num         105217 non-null  int32 
 5   team_id            105217 non-null  int64 
 6   team_city          105217 non-null  object
 7   team_name          105217 non-null  object
 8   team_abbreviation  105217 non-null  object
dtypes: int32(1), int64(3), object(5)
memory usage: 6.8+ MB


In [272]:
join_player_table1 = df_player.merge(df_common_player_info, on=["player_id", "first_name", "last_name"], how="left")
# join_player_table1.info()
#joining df_inactive_players with join_player_table1
join_player_table2 = df_inactive_players.merge(join_player_table1, on=["player_id", "first_name", "last_name","jersey_num","team_id","team_city","team_name","team_abbreviation"], how="left")
join_player_table2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105217 entries, 0 to 105216
Data columns (total 36 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   game_id                           105217 non-null  int64         
 1   player_id                         105217 non-null  int64         
 2   first_name                        105216 non-null  object        
 3   last_name                         105216 non-null  object        
 4   jersey_num                        105171 non-null  object        
 5   team_id                           105217 non-null  int64         
 6   team_city                         105217 non-null  object        
 7   team_name                         105217 non-null  object        
 8   team_abbreviation                 105217 non-null  object        
 9   full_name                         7888 non-null    object        
 10  is_active                       

### Joining(draft_history, draft_combine_history)

In [259]:
df_draft_history = df_draft_history.rename(columns={"person_id":"player_id"})
# df_draft_combine_stats.info()
join_draft_table1 = df_draft_combine_stats.merge(df_draft_history, on=["player_id","season", "player_name"], how="right")
join_draft_table1.head()


,season,player_id,first_name,last_name,player_name,position,height_wo_shoes,height_wo_shoes_ft_in,height_w_shoes,height_w_shoes_ft_in,...,round_pick,overall_pick,draft_type,team_id,team_city,team_name,team_abbreviation,organization,organization_type,player_profile_flag
0,1947,79299,NaN,NaN,Clifton McNeeley,NaN,NaN,NaN,NaN,NaN,...,1,1,Draft,1610610031,Pittsburgh,Ironmen,PIT,Texas-El Paso,College/University,0
1,1947,78109,NaN,NaN,Glen Selbo,NaN,NaN,NaN,NaN,NaN,...,2,2,Draft,1610610035,Toronto,Huskies,HUS,Wisconsin,College/University,1
2,1947,76649,NaN,NaN,Eddie Ehlers,NaN,NaN,NaN,NaN,NaN,...,3,3,Draft,1610612738,Boston,Celtics,BOS,Purdue,College/University,1
3,1947,79302,NaN,NaN,Walt Dropo,NaN,NaN,NaN,NaN,NaN,...,4,4,Draft,1610610032,Providence,Steamrollers,PRO,Connecticut,College/University,0
4,1947,77048,NaN,NaN,Dick Holub,NaN,NaN,NaN,NaN,NaN,...,5,5,Draft,1610612752,New York,Knicks,NYK,Long Island-Brooklyn,College/University,1


### Other table(play_by_play, other_stats, officials, line_score)

In [277]:
# df_line_score.info()
# df_other_stats

## Queries

### Players Table

In [283]:
join_game_table2

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,game_status_text,gamecode,home_team_id,visitor_team_id,season,live_period,live_pc_time,natl_tv_broadcaster_abbreviation,live_period_time_bcast,wh_status
0,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01,HUS vs. NYK,L,0,25.0,...,NaN,19461101/NYKHUS,1.610610e+09,1.610613e+09,1946.0,5.0,NaN,NaN,Q5 -,1.0
1,21946,1610610034,BOM,St. Louis Bombers,24600003,1946-11-02,BOM vs. PIT,W,0,20.0,...,NaN,19461102/PITBOM,1.610610e+09,1.610610e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
2,21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02,PRO vs. BOS,W,0,21.0,...,NaN,19461102/BOSPRO,1.610610e+09,1.610613e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
3,21946,1610610025,CHS,Chicago Stags,24600004,1946-11-02,CHS vs. NYK,W,0,21.0,...,NaN,19461102/NYKCHS,1.610610e+09,1.610613e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
4,21946,1610610028,DEF,Detroit Falcons,24600005,1946-11-02,DEF vs. WAS,L,0,10.0,...,NaN,19461102/WASDEF,1.610610e+09,1.610610e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62363,22022,1610612755,PHI,Philadelphia 76ers,22201016,2023-03-12,PHI vs. WAS,W,240,41.0,...,Final,20230312/WASPHI,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62364,22022,1610612759,SAS,San Antonio Spurs,22201018,2023-03-12,SAS vs. OKC,L,240,33.0,...,Final,20230312/OKCSAS,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62365,22022,1610612740,NOP,New Orleans Pelicans,22201017,2023-03-12,NOP vs. POR,W,240,43.0,...,Final,20230312/PORNOP,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62366,22022,1610612747,LAL,Los Angeles Lakers,22201019,2023-03-12,LAL vs. NYK,L,240,43.0,...,Final,20230312/NYKLAL,1.610613e+09,1.610613e+09,2022.0,4.0,,ESPN,Q4 - ESPN,1.0


In [298]:
team_group = join_game_table2.groupby(["team_abbreviation_home","wl_home"])
team_group["team_abbreviation_home"].count().sort_values(ascending=False)

team_abbreviation_home  wl_home
BOS                     W          2085
NYK                     W          1748
LAL                     W          1741
DET                     W          1501
PHX                     W          1411
                                   ... 
PIT                     W            11
DN                      W            10
NYN                     W            10
CAP                     L            10
AND                     L             9
Name: team_abbreviation_home, Length: 148, dtype: int64

In [301]:
join_game_table2[(join_game_table2["team_abbreviation_home"] == "BOS") & (join_game_table2["wl_home"] == "W")]

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,game_status_text,gamecode,home_team_id,visitor_team_id,season,live_period,live_pc_time,natl_tv_broadcaster_abbreviation,live_period_time_bcast,wh_status
27,21946,1610612738,BOS,Boston Celtics,24600030,1946-11-16,BOS vs. HUS,W,0,18.0,...,NaN,19461116/HUSBOS,1.610613e+09,1.610610e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
58,21946,1610612738,BOS,Boston Celtics,24600059,1946-11-28,BOS vs. PIT,W,0,22.0,...,NaN,19461128/PITBOS,1.610613e+09,1.610610e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
89,21946,1610612738,BOS,Boston Celtics,24600091,1946-12-12,BOS vs. DEF,W,0,29.0,...,NaN,19461212/DEFBOS,1.610613e+09,1.610610e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
95,21946,1610612738,BOS,Boston Celtics,24600095,1946-12-14,BOS vs. PHW,W,0,33.0,...,NaN,19461214/PHWBOS,1.610613e+09,1.610613e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
148,21946,1610612738,BOS,Boston Celtics,24600149,1947-01-10,BOS vs. NYK,W,0,28.0,...,NaN,19470110/NYKBOS,1.610613e+09,1.610613e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62189,22022,1610612738,BOS,Boston Celtics,22200838,2023-02-10,BOS vs. CHA,W,240,44.0,...,Final,20230210/CHABOS,1.610613e+09,1.610613e+09,2022.0,4.0,,ESPN,Q4 - ESPN,1.0
62204,22022,1610612738,BOS,Boston Celtics,22200855,2023-02-12,BOS vs. MEM,W,240,39.0,...,Final,20230212/MEMBOS,1.610613e+09,1.610613e+09,2022.0,4.0,,ABC,Q4 - ABC,1.0
62228,22022,1610612738,BOS,Boston Celtics,22200877,2023-02-15,BOS vs. DET,W,240,49.0,...,Final,20230215/DETBOS,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62283,22022,1610612738,BOS,Boston Celtics,22200935,2023-03-01,BOS vs. CLE,W,240,39.0,...,Final,20230301/CLEBOS,1.610613e+09,1.610613e+09,2022.0,4.0,,ESPN,Q4 - ESPN,1.0


In [359]:
final_2022 = join_game_table2[(join_game_table2["season"] == 2022.0) & (join_game_table2["game_status_text"] == "Final")]
final_date = final_2022["game_date"].idxmax()

final_date = final_2022.loc[final_date, ["team_abbreviation_home","wl_home", "team_abbreviation_away","game_date","matchup_home"]]
print(final_date.to_frame())

                                      62362
team_abbreviation_home                  CHA
wl_home                                   L
team_abbreviation_away                  CLE
game_date               2023-03-12 00:00:00
matchup_home                    CHA vs. CLE


In [409]:
join_game_table2

,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,game_status_text,gamecode,home_team_id,visitor_team_id,season,live_period,live_pc_time,natl_tv_broadcaster_abbreviation,live_period_time_bcast,wh_status
0,21946,1610610035,HUS,Toronto Huskies,24600001,1946-11-01,HUS vs. NYK,L,0,25.0,...,NaN,19461101/NYKHUS,1.610610e+09,1.610613e+09,1946.0,5.0,NaN,NaN,Q5 -,1.0
1,21946,1610610034,BOM,St. Louis Bombers,24600003,1946-11-02,BOM vs. PIT,W,0,20.0,...,NaN,19461102/PITBOM,1.610610e+09,1.610610e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
2,21946,1610610032,PRO,Providence Steamrollers,24600002,1946-11-02,PRO vs. BOS,W,0,21.0,...,NaN,19461102/BOSPRO,1.610610e+09,1.610613e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
3,21946,1610610025,CHS,Chicago Stags,24600004,1946-11-02,CHS vs. NYK,W,0,21.0,...,NaN,19461102/NYKCHS,1.610610e+09,1.610613e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
4,21946,1610610028,DEF,Detroit Falcons,24600005,1946-11-02,DEF vs. WAS,L,0,10.0,...,NaN,19461102/WASDEF,1.610610e+09,1.610610e+09,1946.0,4.0,NaN,NaN,Q4 -,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62363,22022,1610612755,PHI,Philadelphia 76ers,22201016,2023-03-12,PHI vs. WAS,W,240,41.0,...,Final,20230312/WASPHI,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62364,22022,1610612759,SAS,San Antonio Spurs,22201018,2023-03-12,SAS vs. OKC,L,240,33.0,...,Final,20230312/OKCSAS,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62365,22022,1610612740,NOP,New Orleans Pelicans,22201017,2023-03-12,NOP vs. POR,W,240,43.0,...,Final,20230312/PORNOP,1.610613e+09,1.610613e+09,2022.0,4.0,,NaN,Q4 -,1.0
62366,22022,1610612747,LAL,Los Angeles Lakers,22201019,2023-03-12,LAL vs. NYK,L,240,43.0,...,Final,20230312/NYKLAL,1.610613e+09,1.610613e+09,2022.0,4.0,,ESPN,Q4 - ESPN,1.0


In [433]:
winning_home_sum = join_game_table2[["team_name_home", "wl_home","season"]].groupby(["wl_home","season"]).count().sort_values(by="season",ascending=False)
winning_home_sum.head(10)

team_name_home
wl_home season                
W       2022.0             590
L       2022.0             410
        2021.0             555
W       2021.0             653
L       2020.0             480
W       2020.0             575
L       2019.0             465
W       2019.0             571
        2018.0             719
L       2018.0             491